In [1]:
import os
import requests
import sqlite3
import pandas as pd

In [2]:
# directory of the raw data files
_data_root = 'data/data_mod/'

# location of the pipeline metadata store
_pipeline_root = './pipeline/'

In [3]:
import pandas as pd
import tensorflow as tf
import tfx
from tfx.dsl.io import Importer
from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform
import tensorflow_data_validation as tfdv


from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import os
import pprint
pp = pprint.PrettyPrinter()

ImportError: cannot import name 'Importer' from 'tfx.dsl.io' (/opt/conda/lib/python3.7/site-packages/tfx/dsl/io/__init__.py)

In [4]:
# Configuracion de contexto interactivo
context = InteractiveContext(pipeline_root=_pipeline_root)

### Examplex gen

In [5]:
# Instantiate ExampleGen with the input CSV dataset
#example_gen = CsvExampleGen(input_base=_data_root)

example_gen = CsvExampleGen(input_base=_data_root)

In [6]:
# Execute the component
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 50
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

### Estadisticas

In [7]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 51
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [8]:
# Show the output statistics
context.show(statistics_gen.outputs['statistics'])

### Inferir el esquema

In [9]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 52
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [10]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


### Curando el esquema

In [11]:
from tensorflow_metadata.proto.v0 import schema_pb2

try:
    # Get the schema uri
    schema_uri = schema_gen.outputs['schema']._artifacts[0].uri
# for grading since context.run() does not work outside the notebook
except IndexError:
    print("context.run() was no-op")
    schema_path = './pipeline/SchemaGen/schema'
    dir_id = os.listdir(schema_path)[0]
    schema_uri = f'{schema_path}/{dir_id}'
    
schema = tfdv.load_schema_text(os.path.join(schema_uri, 'schema.pbtxt'))



In [12]:
# Modificando Hillshade_9am de 0 a 255
tfdv.set_domain(schema, "Hillshade_9am", schema_pb2.IntDomain(name = "Hillshade_9am", min = 0 , max = 255 ))

# Modificando Hillshade_Noon de 0 a 255
tfdv.set_domain(schema, "Hillshade_Noon", schema_pb2.IntDomain(name = "Hillshade_Noon", min = 0 , max = 255 ))

# Modificando Slope de 0 a 90
tfdv.set_domain(schema, "Slope", schema_pb2.IntDomain(name = "Slope", min = 0 , max = 90 ))

# Modificando Cover_Type de 0 a 6
tfdv.set_domain(schema, "Cover_Type", schema_pb2.StringDomain(name = "Cover_Type", value=['0','1', '2', '3','4','5','6'] ))

In [13]:
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,'Cover_Type'
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"
'Cover_Type',"'0', '1', '2', '3', '4', '5', '6'"


### Entorno de Esquemas

In [14]:
serv_data_df = pd.read_csv("data/data_mod/datos_finales.csv")
serv_data_df = serv_data_df.sample(frac = 0.30, random_state=42) 
serv_data_df = serv_data_df.drop(columns=["Cover_Type"])

# Directorio para guardar los datos de servicio
SERVING_DIR = 'data/serving'
os.makedirs(SERVING_DIR, exist_ok=True)

# Guardar los datos de servicio en un archivo CSV
SERVING_DATA_CSV = os.path.join(SERVING_DIR, 'serving_dataset.csv')
serv_data_df.to_csv(SERVING_DATA_CSV, index=False)

# Definir opciones de estadísticas
stats_options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)

# Calcular estadísticas del conjunto de datos de servicio
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA_CSV, stats_options=stats_options)

# Detectar anomalías en el conjunto de datos de servicio
anomalies = tfdv.validate_statistics(serving_stats, schema=schema)

# Mostrar las anomalías detectadas
tfdv.display_anomalies(anomalies)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


In [15]:
schema.default_environment.append('TRAINING')

### START CODE HERE ###
# Hint: Create another default schema environment with name SERVING (pass in a string)
schema.default_environment.append('SERVING')

# Remove Cover_Type feature from SERVING using TFDV
# Hint: Pass in the strings with the name of the feature and environment 
tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('SERVING')
### END CODE HERE ###

In [16]:
# Validate the serving dataset statistics in the `SERVING` environment
anomalies = tfdv.validate_statistics(serving_stats, schema=schema, environment='SERVING')

# Display the anomalies detected
tfdv.display_anomalies(anomalies)

In [17]:
# Declare the path to the updated schema directory
UPDATED_SCHEMA_DIR = './pipeline/updated_schema'

# Create the said directory
!mkdir -p {UPDATED_SCHEMA_DIR}

# Declare the path to the schema file
schema_file = os.path.join(UPDATED_SCHEMA_DIR, 'schema.pbtxt')

# Save the curated schema to the said file
tfdv.write_schema_text(schema, schema_file)


In [18]:
# Load the schema from the directory we just created
new_schema = tfdv.load_schema_text(schema_file)

# Display the schema. Check that the Domain column still contains the ranges.
tfdv.display_schema(schema=new_schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,'Cover_Type'
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Soil_Type',STRING,required,,'Soil_Type'


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776', 'C5151'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"
'Cover_Type',"'0', '1', '2', '3', '4', '5', '6'"


### Nuevas Estadisticas

In [34]:
### START CODE HERE ###

user_schema_importer = tfx.components.ImportSchemaGen(schema_file=schema_file)
# Run the component
context.run(user_schema_importer, enable_cache=False)

### END CODE HERE ###

context.show(user_schema_importer.outputs['schema'])

AttributeError: module 'tfx.components' has no attribute 'ImportSchemaGen'